In [9]:
from collections import namedtuple
import math

import numpy as np
import z3
from z3 import *

def car(real, imag):
    return complex(real,imag)

def pol(rho, phi):
    return (rho, phi)

def car2pol(car):
    x, y = car.real, car.imag
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    phi = phi / 2 / math.pi * 360
    return(rho, phi)

def pol2car(rho, phi):
    phi = phi / 360 * 2 * math.pi
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return complex(x, y)

In [101]:
def checkcar(var):
    if isinstance(var, tuple):
        print(f'        converting {var} into carthesian to calculate with')
        return pol2car(*var)
    return var

def checkpol(var):
    if isinstance(var, complex):
        print(f'        converting {var} into polar to calculate with')
        return car2pol(var)
    if isinstance(var, int):
        print(f'        int found for {var} assuming angle = 0 for polar calculation')
        return (var, 0)
    return var

def parallel(x1,x2):
    return (x1*x2) / (x1 + x2)

def current(voltage, resistance):
    voltage = checkcar(voltage)
    cur = voltage / resistance
    print(f'    current in polar notation: {car2pol(cur)}')
    return cur

def voltage(current, resistance):
    current = checkcar(current)
    vol = current * resistance
    print(f'    voltage in polar notation: {car2pol(vol)}')
    return vol

def deg2rad(deg):
    return deg / 360 * 2 * math.pi

def rad2deg(rad):
    return rad / 2 / math.pi * 360


def Q(voltage, current):
    voltage = checkpol(voltage)
    current = checkpol(current)
    print(f'    Q = {voltage[0]:.2f} * {current[0]:.2f} * sin({voltage[1]:.2f} - {current[1]:.2f}) WARNING, if current enters at - should be -Q')
    return voltage[0] * current[0] * math.sin(deg2rad(voltage[1] - current[1]))
    
def P(voltage, current):
    voltage = checkpol(voltage)
    current = checkpol(current)
    print(f'    U = {voltage}, I = {current}')
    print(f'    P = {voltage[0]:.2f} * {current[0]:.2f} * cos({voltage[1]:.2f} - {current[1]:.2f}) WARNING, if current enters at - should be -P')
    return voltage[0] * current[0] * math.cos(deg2rad(voltage[1] - current[1]))

def S(P, Q):
    return math.sqrt(P**2 + Q**2)



In [102]:
def capacitor_resistance(capacity, frequency=50):
    return -1 / (2*math.pi * frequency) / capacity

def capacitor_capacity(resistance, frequency=50):
    return 1 / (resistance * 2*math.pi * frequency)
    
def inductor_resistance(capacity, frequency=50):
    return (2*math.pi * frequency) * capacity

def inductor_capacity(resistance, frequency=50):
    return resistance / (2*math.pi*frequency)

In [103]:
def car2complexexpr(cart):
    return ComplexExpr(cart.real, cart.imag)

def pol2complexexpr(size, angle):
    return car2complexexpr(pol2car(size, angle))

def _to_complex(a):
    if isinstance(a, ComplexExpr):
        return a
    else:
        return ComplexExpr(a, RealVal(0))

def _is_zero(a):
    return (isinstance(a, int) and a == 0) or (is_rational_value(a) and a.numerator_as_long() == 0)

class ComplexExpr:
    def __init__(self, r, i):
        self.r = r
        self.i = i

    def __add__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r + other.r, self.i + other.i)

    def __radd__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r + self.r, other.i + self.i)

    def __sub__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r - other.r, self.i - other.i)

    def __rsub__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r - self.r, other.i - self.i)

    def __mul__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r*other.r - self.i*other.i, self.r*other.i + self.i*other.r)

    def __mul__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r*self.r - other.i*self.i, other.i*self.r + other.r*self.i)

    def inv(self):
        den = self.r*self.r + self.i*self.i
        return ComplexExpr(self.r/den, -self.i/den)

    def __div__(self, other):
        inv_other = _to_complex(other).inv()
        return self.__mul__(inv_other)

    def __rdiv__(self, other):
        other = _to_complex(other)
        return self.inv().__mul__(other)

    def __eq__(self, other):
        other = _to_complex(other)
        return And(self.r == other.r, self.i == other.i)

    def __neq__(self, other):
        return Not(self.__eq__(other))

    def simplify(self):
        return ComplexExpr(simplify(self.r), simplify(self.i))

    def repr_i(self):
        if is_rational_value(self.i):
            return "%s*I" % self.i
        else:
            return "(%s)*I" % str(self.i)

    def __repr__(self):
        if _is_zero(self.i):
            return str(self.r)
        elif _is_zero(self.r):
            return self.repr_i()
        else:
            return "%s + %s" % (self.r, self.repr_i())

def Complex(a):
    return ComplexExpr(Real('%s.r' % a), Real('%s.i' % a))
# I = ComplexExpr(RealVal(0), RealVal(1))

def evaluate_cexpr(m, e):
    return ComplexExpr(m[e.r], m[e.i])

def get_ans(m, var):
    r = evaluate_cexpr(m, var).r
    i = evaluate_cexpr(m, var).i
    real = float(r.numerator_as_long())/float(r.denominator_as_long())
    imag = float(i.numerator_as_long())/float(i.denominator_as_long())
    return complex(real, imag)

In [104]:
# template for AC nets
I1 = Complex("x")
I2 = Complex("y")
I3 = Complex("z")
s = Tactic('qfnra-nlsat').solver()





# s.add(ComplexExpr(cur.real, cur.imag) - I1 * ComplexExpr(8,4) - I2*ComplexExpr(0,-6) == 0)
# s.add(I2 * ComplexExpr(0,-6) - ComplexExpr(cur2.real, cur2.imag) == 0)
# s.add(I1 + I3 - I2 == 0)

s.add(100-I1*ComplexExpr(0,6.28)-I3*ComplexExpr(0,-6)==0)
s.add(I3*ComplexExpr(0,-6)+I2*10 == 0)
s.add(I1 + I2 -I3 == 0)
s.check()
m = s.model()

for var in [I1,I2, I3]:
    print(car2pol(get_ans(m,var)))
    

(30.864750979992415, -35.21359368159654)
(15.879783371533687, 85.75016285047698)
(26.466305619222812, -4.249837149523023)


In [105]:
# template for DC nets
I1 = Real('x')
I2 = Real('y')
I3 = Real('z')
I4 = Real('r')
I5 = Real('t')

s = Tactic('qfnra-nlsat').solver()
s.add(-10*I1-100-5*I2==0)
s.add(5*I2 + 100 -20*I4 == 0)
s.add(20*I4 + 40 == 0)

s.add(I1 - I2 - I3 == 0)
s.add(I3 - I4 - I5 == 0)
    
s.check()
m = s.model()

m.eval(I1), m.eval(I2), m.eval(I3),m.eval(I4),m.eval(I5)

(4, -28, 32, -2, 34)

In [106]:
Power = namedtuple('Power', 'Watt VAR')

def power(current, resistance):
    if isinstance(resistance, complex):
        return Power(power(current, resistance.real)[0], 
                     power(current, resistance.imag)[0])
    else:
        p = car2pol(current ** 2 * resistance)[0]
        print(f'    using I2* R: {current}**2 {resistance} = {p}')
        return Power(p,0)


def trans(primaryvoltage, primaryresistance, secondaryresistance, ratio):
    # bring everything to left side
    primaryvoltage = checkcar(primaryvoltage)
    res = primaryresistance + ratio ** 2 * secondaryresistance
    print(f'Total reactance for source {res}')
    Iprim = current(primaryvoltage, res)
    Isec = Iprim * ratio
    
    Uprim = checkcar(primaryvoltage) - primaryresistance * Iprim
    Usec  = Uprim / ratio
    print(f'Transformer: \nIp {car2pol(Iprim)}\nIs {car2pol(Isec)}\nUp {car2pol(Uprim)}\nUs {car2pol(Usec)}')
    print(f'Pprim {P(Uprim, Iprim)}W Qprim {Q(Uprim, Iprim)}VAR')
    psource = P(primaryvoltage, Iprim)
    qsource = Q(primaryvoltage, Iprim)
    print(f'Powers on primary side Psource: {psource}, Qsource {qsource}')
    print(f'Resistances on primary side: {power(Iprim, primaryresistance)}')
    print(f'Resistances on secondary side: {power(Isec, secondaryresistance)}')
    print(f'Efficiency transformer: {power(Isec, secondaryresistance).Watt} / {psource} = {power(Isec, secondaryresistance).Watt / psource}')

trans((100,0), 2, (100-200j), 0.1)

        converting (100, 0) into carthesian to calculate with
Total reactance for source (3-2.0000000000000004j)
    current in polar notation: (27.735009811261456, 33.6900675259798)
Transformer: 
Ip (27.735009811261456, 33.6900675259798)
Is (2.7735009811261455, 33.6900675259798)
Up (62.017367294604234, -29.744881296942225)
Us (620.1736729460423, -29.74488129694223)
        converting (53.846153846153854-30.769230769230774j) into polar to calculate with
        converting (23.076923076923073+15.384615384615387j) into polar to calculate with
    U = (62.017367294604234, -29.744881296942225), I = (27.735009811261456, 33.6900675259798)
    P = 62.02 * 27.74 * cos(-29.74 - 33.69) WARNING, if current enters at - should be -P
        converting (53.846153846153854-30.769230769230774j) into polar to calculate with
        converting (23.076923076923073+15.384615384615387j) into polar to calculate with
    Q = 62.02 * 27.74 * sin(-29.74 - 33.69) WARNING, if current enters at - should be -Q
Ppr

In [107]:
nullast(35, 230,0.3), kortsluit(2300, 0.14*230, 30000/230*0.9)

Nullast:
S = 69.0
Q = 59.464274989274024 : sqrt(S**2 - P**2) = sqrt(4761.0 - 1225)
phi = 59.51941427439488 : rad2deg(acos((P / (U * I))))
Rij = 1511.4285714285713 : U**2 / P
Xmu = 889.6097700601232 : U**2 / Q

Kortsluit:
S = 3780.0000000000005
Q = 2999.7333214804285 : sqrt(S**2 - P**2) = sqrt(14288400.000000004 - 5290000)
phi = 52.521361030590604 : rad2deg(acos((P / (U * I))))
Rk = 0.16689986282578875 : P / I ** 2. Note Rk = R1 + R'2
Xk = 0.21767612168218745 : Q / I ** 2



((1511.4285714285713, 889.6097700601232),
 (0.16689986282578875, 0.21767612168218745))

In [108]:
trans(230, 0.1668 + 0.2176j, 4000, 230/10000)


Total reactance for source (2.2827999999999995+0.2176j)
    current in polar notation: (100.29882372039609, -5.445069764447646)
Transformer: 
Ip (100.29882372039609, -5.445069764447646)
Is (2.30687294556911, -5.445069764447646)
Up (212.232310992358, -5.445069764447647)
Us (9227.491782276436, -5.445069764447647)
        converting (211.27464017670655-20.139022999146384j) into polar to calculate with
        converting (99.84623826876495-9.517496691468047j) into polar to calculate with
    U = (212.232310992358, -5.445069764447647), I = (100.29882372039609, -5.445069764447646)
    P = 212.23 * 100.30 * cos(-5.45 - -5.45) WARNING, if current enters at - should be -P
        converting (211.27464017670655-20.139022999146384j) into polar to calculate with
        converting (99.84623826876495-9.517496691468047j) into polar to calculate with
    Q = 212.23 * 100.30 * sin(-5.45 - -5.45) WARNING, if current enters at - should be -Q
Pprim 21286.651147994795W Qprim -3.2997795540973475e-13VAR
   

### Asynchronous machines

In [109]:
from math import sqrt
def phase2linevoltage(phasevoltage):
    return sqrt(3) * phasevoltage
def line2phasevoltage(linevoltage):
    return linevoltage / sqrt(3)
line2phasevoltage(10000)

def S3f(Ul, I):
    return sqrt(3) * Ul * I

def Pdriehoek(Pster):
    return 3 * Pster

In [110]:
def nullast(P,U,I):
    S = U * I # schijnbaar vermogen
    Q = (S**2 - P**2)**0.5 # blind vermogen
    # nullasten staan parallel aan de bron dus P = U**2 / R gebruiken
    Rij = U**2 / P  
    Xmu = U**2 / Q
    phi = rad2deg(math.acos((P / (U * I)))) # from formula P = U * I * cos(phi)
    print(
        'Nullast:\n'
        f'S = {S}\n'
        f'Q = {Q} : sqrt(S**2 - P**2) = sqrt({S**2} - {P**2})\n'
        f'phi = {phi} : rad2deg(acos((P / (U * I))))\n'
        f'Rij = {Rij} : U**2 / P\n'
        f'Xmu = {Xmu} : U**2 / Q\n')
    return Rij, Xmu


def kortsluit(P,U,I):
    S = U * I # schijnbaar vermogen
    Q = (S**2 - P**2)**0.5 # blind vermogen
    # nullasten staan in serie met bron dus P = I**2 * R gebruiken
    Rk = P / I**2
    Xk = Q / I**2
    phi = rad2deg(math.acos((P / (U * I)))) # from formula P = U * I * cos(phi)
    print(
        'Kortsluit:\n'
        f'S = {S}\n'
        f'Q = {Q} : sqrt(S**2 - P**2) = sqrt({S**2} - {P**2})\n'
        f'phi = {phi} : rad2deg(acos((P / (U * I))))\n'
        f"Rk = {Rk} : P / I ** 2. Note Rk = R1 + R'2\n"
        f'Xk = {Xk} : Q / I ** 2\n')
    return Rk, Xk



In [111]:
def slip(fnet, frotor):
    # slip = 1, means rotor is stationary
    # slip = 0, means fnet == frotor
    # slip < 0, means machine is acting as generator
    return (fnet - frotor) / fnet
slip(50,0), slip(50,50), slip(50, 60)

def slip2freq(slip, fnet=None):
    if not fnet: fnet = 50
    # slip = (fnet - frot) / fnet
    return -(slip * fnet - fnet)
slip2freq(0.0212)

48.94

In [112]:
def R2(Rk, ohmmeasured):
    # vervangingsweerstand as berekenen bij bekend zijn van Rk vanuit kortsluitproef
    return Rk - (ohmmeasured / 2)

def Ras(R2, slip):
    # vervangingsweerstand as berekenen inclusief slip
    return R2 * (1-slip) / slip

def Pas(I, R2, s):
    # asvermogen bepalen
    return 3 * I**2 * R2 * (1-s) / s

def koppel(P, freq):
    # P = w T
    return P / (freq * 2 * math.pi)

In [114]:
def asynchrone(voltage, Xk, Rk, R2, slip):
    # gegeven vervangingsschema en een slip bepaal stroom, vermogens en rendement
    if isinstance(voltage, int):
        voltage = (voltage, 0)
    reactance = complex(Rk + Ras(R2, slip), Xk)
    cur = checkpol(current(voltage, reactance))
    print('Asynchronous machine')
    print('Total reactance = ', reactance)
    print(f'Current = {cur} (voltage / reactance)')
    print('P = ', 3* P(voltage, cur))
    print('Q = ', 3*Q(voltage, cur))
    print('Pas = ', Pas(cur[0], R2, slip))
    print('rendement bij slip > 0 (anders 1/x doen)', Pas(cur[0], R2, slip) / (3 * P(voltage, cur)))
    return Pas(cur[0], R2, slip)
    
    
asynchrone((230,0), 0.2, 0.206, 0.156, -0.033333)

        converting (230, 0) into carthesian to calculate with
    current in polar notation: (49.62924367654542, -177.52658296177944)
        converting (-49.583006676742166-2.141792893831239j) into polar to calculate with
Asynchronous machine
Total reactance =  (-4.630046800468004+0.2j)
Current = (49.62924367654542, -177.52658296177944) (voltage / reactance)
    U = (230, 0), I = (49.62924367654542, -177.52658296177944)
    P = 230.00 * 49.63 * cos(0.00 - -177.53) WARNING, if current enters at - should be -P
P =  -34212.274606952094
    Q = 230.00 * 49.63 * sin(0.00 - -177.53) WARNING, if current enters at - should be -Q
Q =  1477.8370967435362
Pas =  -35734.44681659795
    U = (230, 0), I = (49.62924367654542, -177.52658296177944)
    P = 230.00 * 49.63 * cos(0.00 - -177.53) WARNING, if current enters at - should be -P
rendement bij slip > 0 (anders 1/x doen) 1.0444919908755947


-35734.44681659795

In [115]:
S1f = 60000 / 3
nominalcurrent = S1f / 230
nullast(30, 230, 0.2), kortsluit(1000, 20, 0.8*nominalcurrent)

Nullast:
S = 46.0
Q = 34.87119154832539 : sqrt(S**2 - P**2) = sqrt(2116.0 - 900)
phi = 49.29429316895856 : rad2deg(acos((P / (U * I))))
Rij = 1763.3333333333333 : U**2 / P
Xmu = 1517.0115402190895 : U**2 / Q

Kortsluit:
S = 1391.3043478260872
Q = 967.3302374472607 : sqrt(S**2 - P**2) = sqrt(1935727.788279774 - 1000000)
phi = 44.04862567408433 : rad2deg(acos((P / (U * I))))
Rk = 0.20664062499999994 : P / I ** 2. Note Rk = R1 + R'2
Xk = 0.19988972484750028 : Q / I ** 2



((1763.3333333333333, 1517.0115402190895),
 (0.20664062499999994, 0.19988972484750028))

In [116]:
R2(0.20664,0.1), slip2freq(0.05, 50)

(0.15664, 47.5)

In [117]:
asynchrone((230,0), 0.199, 0.20, 0.1566, 0.05)

        converting (230, 0) into carthesian to calculate with
    current in polar notation: (72.29000149436987, -3.585994993268026)
        converting (72.14846104871398-4.521491386500625j) into polar to calculate with
Asynchronous machine
Total reactance =  (3.1754+0.199j)
Current = (72.29000149436987, -3.585994993268026) (voltage / reactance)
    U = (230, 0), I = (72.29000149436987, -3.585994993268026)
    P = 230.00 * 72.29 * cos(0.00 - -3.59) WARNING, if current enters at - should be -P
P =  49782.43812361264
    Q = 230.00 * 72.29 * sin(0.00 - -3.59) WARNING, if current enters at - should be -Q
Q =  3119.829056685431
Pas =  46646.93153397905
    U = (230, 0), I = (72.29000149436987, -3.585994993268026)
    P = 230.00 * 72.29 * cos(0.00 - -3.59) WARNING, if current enters at - should be -P
rendement bij slip > 0 (anders 1/x doen) 0.937015809031933


46646.93153397905

In [19]:
nullast(180/3, 230, 0.4)

Nullast:
S = 92.0
Q = 69.74238309665078
phi = 49.29429316895856
Rij = 881.6666666666666
Xij = 758.5057701095448



(881.6666666666666, 758.5057701095448)

Kortsluit:
S = 899.76
Q = 748.0428180258133
phi = 56.24079944551819
Rij = 0.08167935898561818
Xij = 0.12219931574028771



(0.08167935898561818, 0.12219931574028771)

In [21]:
asynchrone()

48.94

        converting (230, 0) into carthesian to calculate with
    current in polar notation: (49.62924367654542, -177.52658296177944)
        converting (-49.583006676742166-2.141792893831239j) into polar to calculate with
total reactance (-4.630046800468004+0.2j)
current (49.62924367654542, -177.52658296177944)
(230, 0) (49.62924367654542, -177.52658296177944)
    P = 230.00 * 49.63 * cos(0.00 - -177.53) WARNING, if current enters at - should be -P
vermogen -34212.274606952094
    Q = 230.00 * 49.63 * sin(0.00 - -177.53) WARNING, if current enters at - should be -Q
blindvermogen 1477.8370967435362
asvermogen -35734.44681659795
(230, 0) (49.62924367654542, -177.52658296177944)
    P = 230.00 * 49.63 * cos(0.00 - -177.53) WARNING, if current enters at - should be -P
rendement bij slip > 0 (anders 1/x doen) 1.0444919908755947


-35734.44681659795

### Synchnonous machines

In [24]:
def X1(S1f, Unom, I2nullast, I1kortsluit, I2kortsluit):
    lam = Unom / I2nullast
    print(f'Bij een I2 van {I2nullast}A wordt de nominale fasespanning gehaald van {Unom}V')
    I1nullast = I1kortsluit / I2kortsluit * I2nullast
    print(f'Bij een I2 van {I2kortsluit}A treedt een I1 op van {I1kortsluit}A, dus bij I2 van {I2nullast}A is I1 {I1nullast}A')
    X1 = Unom / I1nullast
    print(f'X1 is daarom {Unom} / {I1nullast} = {X1}ohm')
    return X1
Imax = 20
x1 = X1(56679/3, 230, 2.3, Imax*0.8, 9.2)

Bij een I2 van 2.3A wordt de nominale fasespanning gehaald van 230V
Bij een I2 van 9.2A treedt een I1 op van 16.0A, dus bij I2 van 2.3A is I1 4.0A
X1 is daarom 230 / 4.0 = 57.5ohm


In [25]:
def U12(Unet, I1, X1, l = None):
    UX1 = voltage(I1, complex(0,X1))
    print(f'Voltage over X1 {car2pol(UX1)}')
    u12 = car2pol(pol2car(*Unet) - UX1)
    print(f'Unet - UX1 = U12 --> {Unet} - {car2pol(UX1)} = {u12}')
    print(f'lasthoek is {u12[1] - Unet[1]}')
    print(f'Pas is {3 * -P(u12, I1)}')
    QX1 = 3 * I1[0]**2 * X1
    Qas = 3 * Q(u12, I1)
    print(f'Qas is {Qas}')
    print(f'QX1 is {QX1}')
    print(f'Pnet is {-3 * P(Unet, I1)}')
    print(f'Qnet is {-3 * Q(Unet, I1)}')
    machineQ = QX1 + Qas
    behavior = 'inductive' if machineQ > 0 else 'capacitive'
    print(f'machine is operating {machineQ}, therefore {behavior}')
    
    
    if l:
        I2 = u12[0] / l
        print(f'I2 is {I2}A')
    return u12

In [26]:
unet = (230,0)
z = complex(3,2)
inet = car2pol(pol2car(*unet) / z)
i1 = inet[0], inet[1] - 180

In [27]:
U12(unet, (50,20), 9.86,57.5)

        converting (50, 20) into carthesian to calculate with
    voltage in polar notation: (492.99999999999994, 110.00000000000001)
Voltage over X1 (492.99999999999994, 110.00000000000001)
Unet - UX1 = U12 --> (230, 0) - (492.99999999999994, 110.00000000000001) = (611.156549587252, -49.28989383753886)
lasthoek is -49.28989383753886
    P = 611.16 * 50.00 * cos(-49.29 - 20.00) WARNING, if current enters at - should be -P
Pas is -32419.39541711384
    Q = 611.16 * 50.00 * sin(-49.29 - 20.00) WARNING, if current enters at - should be -Q
Qas is -85749.69494473559
QX1 is 73950.0
    P = 230.00 * 50.00 * cos(0.00 - 20.00) WARNING, if current enters at - should be -P
Pnet is -32419.395417113843
    Q = 230.00 * 50.00 * sin(0.00 - 20.00) WARNING, if current enters at - should be -Q
Qnet is 11799.694944735571
machine is operating -11799.694944735587, therefore capacitive
I2 is 10.628809558039165A


(611.156549587252, -49.28989383753886)

In [28]:
from functools import partial
p = partial(asynchrone, 233, 78, 45, 10)

In [29]:
import numpy as np
s = np.linspace(-0.99,0.99,20)
net = 50


def tas(slip):
    was = 2 * math.pi * net * (1-slip)
    print(slip)
    return p(slip) / was
t = [tas(x) for x in s]

-0.99
        converting (233, 0) into carthesian to calculate with
    current in polar notation: (2.8457072716371075, -72.29599041025817)
        converting (0.8653788071255943-2.710935151571537j) into polar to calculate with
total reactance (24.8989898989899+78j)
current (2.8457072716371075, -72.29599041025817)
    P = 233.00 * 2.85 * cos(0.00 - -72.30) WARNING, if current enters at - should be -P
vermogen 604.8997861807899
    Q = 233.00 * 2.85 * sin(0.00 - -72.30) WARNING, if current enters at - should be -Q
blindvermogen 1894.9436709485044
asvermogen -488.3369470587314
    P = 233.00 * 2.85 * cos(0.00 - -72.30) WARNING, if current enters at - should be -P
rendement bij slip > 0 (anders 1/x doen) -0.8073022312373233
-0.8857894736842105
        converting (233, 0) into carthesian to calculate with
    current in polar notation: (2.8580472598977225, -73.09164552846183)
        converting (0.831239342042167-2.7344972656870925j) into polar to calculate with
total reactance (23.7106357

In [30]:
import altair as alt
import numpy as np
import pandas as pd


source = pd.DataFrame({
  's': s,
  't': t
})

alt.Chart(source).mark_line().encode(
    x='s',
    y='t'
)

alt.Chart(...)